In [19]:
import os
import re
import json
import random
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist
from scipy.optimize import linear_sum_assignment

# Pyclustering dependencies
from pyclustering.cluster.kmedoids import kmedoids
from pyclustering.utils.metric import distance_metric, type_metric

# Global seed
SEED = 42
random.seed(SEED)

# Folder structure configurations
RESULTS_DIR = "results_17_01"   # JSON input
RESULTS_CSV_DIR = "results_csv_clean_2" # Output root for CSVs and clustering
os.makedirs(RESULTS_CSV_DIR, exist_ok=True)

# Known folder “labels”
S_FOLDERS   = ["s_8", "s_15", "s_25"]
DISTANCES   = ["smooth", "uniform", "hamming", "overall"]
NOISE_LEVELS = ["n_0.02", "n_0.04", "n_0.06"]

# Map s_x to # of clusters
CLUSTER_MAPPING = {
    "s_8": 8,
    "s_15": 15,
    "s_25": 25
}


In [20]:
def parse_filename(filename, s_values, distances, noise_values):
    """
    Parse the filename to find:
      - which s_folder (e.g., 's_8', 's_15', 's_25')
      - which distance ('smooth', 'uniform', 'hamming', 'overall')
      - which noise level ('n_0.02', 'n_0.04', 'n_0.06')

    Returns: (s_folder, distance, noise)

    Raises ValueError if anything is missing or not found.
    """
    # 1. Find s_folder
    s_folder = None
    for s_val in s_values:
        if s_val in filename:
            s_folder = s_val
            break
    if not s_folder:
        raise ValueError(f"Could not parse s_folder (s_8, s_15, s_25) from {filename}")

    # 2. Find distance
    distance = None
    for dist in distances:
        if dist in filename:
            distance = dist
            break
    # If nothing found, default to "smooth"
    if not distance:
        distance = "smooth"

    # 3. Find noise level
    noise = None
    for nv in noise_values:
        if nv in filename:
            noise = nv
            break
    if not noise:
        raise ValueError(f"Could not parse noise level (n_0.02, n_0.04, n_0.06) from {filename}")

    return s_folder, distance, noise


In [21]:
def load_data(json_path):
    """
    Loads JSON data into a dictionary of DataFrames/Series
    (similar to your existing logic).
    """
    with open(json_path, 'r') as file:
        data = json.load(file)

    parsed_data = {}
    for key, value in data.items():
        if isinstance(value, list):
            try:
                arr = np.array(value)
                if arr.ndim == 1:
                    parsed_data[key] = pd.Series(value)
                elif arr.ndim == 2:
                    parsed_data[key] = pd.DataFrame(value)
                else:
                    # If it's a list of lists-of-lists, convert each slice
                    parsed_data[key] = {
                        i: pd.DataFrame(subarr) for i, subarr in enumerate(arr)
                    }
            except ValueError:
                # Fallback if direct array conversion fails
                parsed_data[key] = {
                    i: pd.DataFrame(subarr) for i, subarr in enumerate(value) if isinstance(subarr, list)
                }
        else:
            parsed_data[key] = value

    return parsed_data


def create_folder_structure():
    """
    Create the 3-level folder structure under RESULTS_CSV_DIR:
        s_* / distance / n_*
    But do not create subfolders yet for Dw/Dkl; that happens dynamically.
    """
    for s_val in S_FOLDERS:
        s_path = os.path.join(RESULTS_CSV_DIR, s_val)
        os.makedirs(s_path, exist_ok=True)
        for dist in DISTANCES:
            dist_path = os.path.join(s_path, dist)
            os.makedirs(dist_path, exist_ok=True)
            for nv in NOISE_LEVELS:
                noise_path = os.path.join(dist_path, nv)
                os.makedirs(noise_path, exist_ok=True)


def process_json_files():
    """
    For each JSON in RESULTS_DIR:
      1) parse s_folder, distance, noise
      2) load JSON data
      3) create subfolder named after the filename (minus .json)
      4) for each key in 'parsed_data' that starts with 'all_', create a
         corresponding sub-subfolder and save each matrix to CSV.
    """
    for filename in os.listdir(RESULTS_DIR):
        if not filename.endswith(".json"):
            continue
        
        # parse s_folder, distance, noise
        s_folder, distance, noise = parse_filename(
            filename,
            s_values=S_FOLDERS,
            distances=DISTANCES,
            noise_values=NOISE_LEVELS
        )
        
        # load data
        json_path = os.path.join(RESULTS_DIR, filename)
        parsed_data = load_data(json_path)

        # build the subfolder path: results_csv/s_folder/distance/noise/<run_name>/
        run_name = os.path.splitext(filename)[0]
        run_path = os.path.join(RESULTS_CSV_DIR, s_folder, distance, noise, run_name)
        os.makedirs(run_path, exist_ok=True)

        # For each key in parsed_data that starts with "all_", create a subfolder
        for key, val in parsed_data.items():
            if not key.startswith("all_"):
                continue

            # e.g., key = "all_Dw" => subfolder name = "Dw"
            subfolder_name = key.replace("all_", "")
            subfolder_path = os.path.join(run_path, subfolder_name)
            os.makedirs(subfolder_path, exist_ok=True)

            # val could be a dict of DataFrames or a single DataFrame
            # Often in your code it's a dict: { i: DataFrame, i: DataFrame, ... }
            # So handle that scenario:
            if isinstance(val, dict):
                # Save each matrix to CSV
                for i, df in val.items():
                    csv_path = os.path.join(subfolder_path, f"{subfolder_name}_{i}.csv")
                    df.to_csv(csv_path, index=False, header=False)
            elif isinstance(val, pd.DataFrame):
                # Single DF scenario
                csv_path = os.path.join(subfolder_path, f"{subfolder_name}.csv")
                val.to_csv(csv_path, index=False, header=False)
            else:
                # Possibly a Series or something else
                # Adjust if needed
                pass


In [22]:
def pam_cosine_clustering(data, n_clusters, random_seed=SEED):
    """
    Performs k-medoids (PAM) with cosine distance.
    data shape expected: (num_bins, num_signatures).
    Returns: (clusters, medoid_signatures)
    """
    # Convert to NumPy
    if isinstance(data, pd.DataFrame):
        data = data.to_numpy()

    # Transpose => (num_signatures, num_bins)
    data = data.T
    num_signatures, _ = data.shape

    if n_clusters > num_signatures:
        raise ValueError(f"Requested {n_clusters} clusters, but only {num_signatures} signatures available.")

    # Define custom distance
    def custom_cosine_distance(a, b):
        return cdist([a], [b], "cosine")[0, 0]

    metric = distance_metric(type_metric.USER_DEFINED, func=custom_cosine_distance)
    initial_medoids = random.sample(range(num_signatures), n_clusters)

    kmed = kmedoids(data.tolist(), initial_medoids, metric=metric)
    kmed.process()
    clusters = kmed.get_clusters()
    medoids = kmed.get_medoids()
    medoid_signatures = np.array([data[idx] for idx in medoids])
    return clusters, medoid_signatures

def load_and_cluster(folder_path, n_clusters, output_csv_path, random_seed=SEED):
    """
    Loads CSV files in `folder_path`, concatenates them, does PAM clustering,
    and saves the medoids to `output_csv_path`.
    """
    csv_files = [f for f in os.listdir(folder_path) if f.endswith(".csv")]
    if not csv_files:
        print(f"No CSV files in {folder_path}, skipping.")
        return None, None

    matrices = []
    for f in csv_files:
        mat = pd.read_csv(os.path.join(folder_path, f), header=None)
        matrices.append(mat)

    combined_data = pd.concat(matrices, axis=1)  # horizontally
    clusters, medoids = pam_cosine_clustering(combined_data, n_clusters, random_seed)

    # Save medoids
    pd.DataFrame(medoids.T).to_csv(output_csv_path, index=False, header=False)
    print(f"Medoid signatures saved to: {output_csv_path}")
    
    return clusters, medoids






results_csv /
    s_15 /
        uniform /
            n_0.02 /
                <run_name> /
                    Dw /
                    Dkl /
                    ...


In [23]:
def traverse_folders(root_dir, s_values, distances, noise_values):
    """
    Yields (s_folder, distance, noise, run_folder, subfolder_path)
    for each folder that actually exists under:
        root_dir / s_folder / distance / noise / run_folder / ...
    We'll pick up any sub-subfolders under run_folder.
    """
    for s_val in s_values:
        s_path = os.path.join(root_dir, s_val)
        if not os.path.exists(s_path):
            continue
        for dist in distances:
            dist_path = os.path.join(s_path, dist)
            if not os.path.exists(dist_path):
                continue
            for nv in noise_values:
                noise_path = os.path.join(dist_path, nv)
                if not os.path.exists(noise_path):
                    continue
                # each run_folder is the JSON-based folder name
                for run_folder in os.listdir(noise_path):
                    run_path = os.path.join(noise_path, run_folder)
                    if not os.path.isdir(run_path):
                        continue
                    # now yield any subfolder inside run_path
                    for subfolder in os.listdir(run_path):
                        subfolder_path = os.path.join(run_path, subfolder)
                        if os.path.isdir(subfolder_path):
                            yield (s_val, dist, nv, run_folder, subfolder_path)


In [24]:
def run_clustering():
    for s_val, dist, noise, run_name, subfolder_path in traverse_folders(
        root_dir=RESULTS_CSV_DIR,
        s_values=S_FOLDERS,
        distances=DISTANCES,
        noise_values=NOISE_LEVELS
    ):
        # Filter so we only process "Dw" or "Dkl" subfolders
        subfolder_name = os.path.basename(subfolder_path)
        if subfolder_name not in ("Dw", "Dkl"):
            continue

        n_clusters = CLUSTER_MAPPING[s_val]
        output_csv = os.path.join(subfolder_path, f"medoids_{subfolder_name}.csv")

        print(f"Clustering {subfolder_path} -> {output_csv} with k={n_clusters}")
        clusters, medoids = load_and_cluster(
            folder_path=subfolder_path,
            n_clusters=n_clusters,
            output_csv_path=output_csv,
            random_seed=SEED
        )

In [25]:
import re

# Suppose these are the known JSON filenames for each s_x:
file_map = {
    "s_8":  "Results_20250117_154939_overall_s_8_n_0.02__GRCh37_98_86_39_22a_43_96_37_54.json",
    "s_15": "Results_20250117_154127__s_15_n_0.02__GRCh37_98_86_39_22a_43_96_37_54_87_17b_21_99_3_33_93.json",
    "s_25": "Results_20250117_154536__s_25_n_0.02__GRCh37_98_86_39_22a_43_96_37_54_87_17b_21_99_3_33_93_26_12_22b_17a_9_25_55_89_13_46"
}

def save_ground_truth_signatures(cosmic_signatures_path):
    """
    Loads the cosmic signatures, extracts the relevant SBS IDs from each JSON
    file name (the portion after GRCh37_), and saves them to CSV in their
    corresponding s_x folder.
    """
    cosmic_df = pd.read_csv(cosmic_signatures_path, sep="\t")
    # If there's a column "Type", set as index
    if "Type" in cosmic_df.columns:
        cosmic_df.set_index("Type", inplace=True)

    # For each s_folder + known JSON filename:
    for s_val, fname in file_map.items():
        match = re.search(r"GRCh37_([\w\d_]+)", fname)
        if not match:
            print(f"No signature portion found in {fname}, skipping.")
            continue
        sig_portion = match.group(1)
        sbs_ids = sig_portion.split("_")  # like ["98","86","39","22a",...]
        # Prepend "SBS" to each id
        # watch out if you have "22a" or "17b" etc. — adjust if needed
        signature_names = [f"SBS{id_}" for id_ in sbs_ids]

        # Filter cosmic
        filtered_sigs = cosmic_df[signature_names]

        # Save to CSV
        out_path = os.path.join(RESULTS_CSV_DIR, s_val, f"ground_truth_signatures_{s_val}.csv")
        filtered_sigs.to_csv(out_path, index=True)
        print(f"Saved ground truth to {out_path}")



In [ ]:
# 1) Create the standard 3-level directory 
create_folder_structure()

# 2) Convert JSON => CSV subfolders
process_json_files()

# 3) Run clustering on each subfolder
run_clustering()




In [41]:

def evaluation(true_sigs, est_sigs, cutoff=0.9, dist="cos"):
    """
    Evaluate extracted sigs (est_sigs) vs. true_sigs.
    Returns a tuple with (n_ground_truth, n_detected, tp, fp, fn, precision, recall, f1, index_map).
    """
    if true_sigs.shape[1] >= est_sigs.shape[1]:
        mat1, mat2 = est_sigs, true_sigs
    else:
        mat1, mat2 = true_sigs, est_sigs

    if dist == "cos":
        con_mat = cdist(mat1.T, mat2.T, "cosine")
    elif dist == "cor":
        con_mat = cdist(mat1.T, mat2.T, "correlation")
    else:
        raise ValueError("Distance must be 'cos' or 'cor'")

    # Hungarian assignment
    row_ind, col_ind = linear_sum_assignment(con_mat)
    similarity_mat = 1 - con_mat  # convert distance -> similarity

    idx_map = {}
    tp = 0
    for r, c in zip(row_ind, col_ind):
        idx_map[r] = c
        if similarity_mat[r, c] >= cutoff:
            tp += 1

    computed_fp = mat1.shape[1] - tp
    computed_fn = computed_fp
    if true_sigs.shape[1] >= est_sigs.shape[1]:
        base_fp = 0
        base_fn = true_sigs.shape[1] - est_sigs.shape[1]
    else:
        base_fp = est_sigs.shape[1] - true_sigs.shape[1]
        base_fn = 0

    fp = base_fp + computed_fp
    fn = base_fn + computed_fn

    n_gt = true_sigs.shape[1]
    n_detected = est_sigs.shape[1]

    try:
        precision = round(tp / (tp + fp), 2)
        recall    = round(tp / (tp + fn), 2)
        f1_score  = round(2 * precision * recall / (precision + recall), 2)
    except ZeroDivisionError:
        precision, recall, f1_score = 0, 0, 0

    return (n_gt, n_detected, tp, fp, fn, precision, recall, f1_score, idx_map)

def run_evaluation(cutoff=0.9):
    """
    For each subfolder with medoids_<subfolder>.csv,
    read the corresponding ground truth, evaluate, save results in .txt,
    but ONLY for 'Dw' or 'Dkl'.
    """
    for s_val, dist, noise, run_name, subfolder_path in traverse_folders(
        root_dir=RESULTS_CSV_DIR,
        s_values=S_FOLDERS,
        distances=DISTANCES,
        noise_values=NOISE_LEVELS
    ):
        subfolder_name = os.path.basename(subfolder_path)
        
        # Skip any subfolder that is not Dw or Dkl
        if subfolder_name not in ("Dw", "Dkl"):
            continue
        
        gt_csv = os.path.join(RESULTS_CSV_DIR, s_val, f"ground_truth_signatures_{s_val}.csv")
        if not os.path.exists(gt_csv):
            continue
        
        true_signatures = pd.read_csv(gt_csv, index_col=0).to_numpy()
        
        # Medoids file path depends on subfolder name (Dw or Dkl)
        medoids_csv = os.path.join(subfolder_path, f"medoids_{subfolder_name}.csv")
        if not os.path.exists(medoids_csv):
            continue
        
        est_signatures = pd.read_csv(medoids_csv, header=None).to_numpy()
        
        # Run evaluation
        eval_res = evaluation(
            true_sigs=true_signatures,
            est_sigs=est_signatures,
            cutoff=cutoff,
            dist="cos"
        )
        
        # Write results
        eval_txt = os.path.join(subfolder_path, f"evaluation_{subfolder_name}.txt")
        with open(eval_txt, "w") as f:
            f.write("=== Evaluation Results ===\n")
            f.write(f"Cutoff: {cutoff}\n")
            f.write(f"s_folder: {s_val}\n")
            f.write(f"distance: {dist}\n")
            f.write(f"noise: {noise}\n")
            f.write(f"run_folder: {run_name}\n")
            f.write(f"subfolder: {subfolder_name}\n")
            f.write(f"Number of Ground Truth: {eval_res[0]}\n")
            f.write(f"Number of Detected: {eval_res[1]}\n")
            f.write(f"True Positives: {eval_res[2]}\n")
            f.write(f"False Positives: {eval_res[3]}\n")
            f.write(f"False Negatives: {eval_res[4]}\n")
            f.write(f"Precision: {eval_res[5]}\n")
            f.write(f"Recall: {eval_res[6]}\n")
            f.write(f"F1 Score: {eval_res[7]}\n")
            f.write("Index Pairs (Extracted -> Ground Truth):\n")
            for e_idx, t_idx in eval_res[8].items():
                f.write(f"  Extracted {e_idx} -> Ground Truth {t_idx}\n")

        print(f"Evaluation written to {eval_txt}")


In [39]:


# EVALUATION WITHOUT HUNGARIAN RE-ARRANGMENT

def evaluation_figS1(true_sigs, est_sigs, cutoff=0.9, dist="cos"):
    """
    Evaluate extracted sigs (est_sigs) vs. true_sigs in a “Figure S1” style:
      - Each extracted signature is TP if it matches (>= cutoff) at least one ground-truth signature.
      - Otherwise it is FP.
      - A ground-truth signature is FN if no extracted signature matches it above the cutoff.
    Returns:
      n_ground_truth, n_detected, tp, fp, fn, precision, recall, f1_score
    """
    # Number of ground-truth and extracted signatures
    n_gt = true_sigs.shape[1]
    n_detected = est_sigs.shape[1]

    # Compute pairwise distance matrix => then convert to similarity
    if dist == "cos":
        D = cdist(est_sigs.T, true_sigs.T, metric="cosine")  # shape: (n_detected, n_gt)
    elif dist == "cor":
        D = cdist(est_sigs.T, true_sigs.T, metric="correlation")
    else:
        raise ValueError("Distance must be 'cos' or 'cor'")

    similarity = 1 - D  # Convert distance to similarity if using cosine/correlation

    # (1) Classify each extracted signature
    # max similarity to any ground-truth signature
    max_sim_extracted = np.max(similarity, axis=1)  # shape: (n_detected,)
    # True positives are those that exceed the cutoff
    tp = np.sum(max_sim_extracted >= cutoff)
    # False positives are those that do not match any ground-truth signature above cutoff
    fp = n_detected - tp

    # (2) Classify each ground-truth signature
    # max similarity of each ground-truth signature to any extracted signature
    max_sim_truth = np.max(similarity, axis=0)  # shape: (n_gt,)
    # False negatives are ground-truth signatures not matched by any extracted signature
    fn = np.sum(max_sim_truth < cutoff)

    # (3) Precision/Recall/F1
    try:
        precision = tp / (tp + fp)
    except ZeroDivisionError:
        precision = 0.0
    try:
        recall = tp / (tp + fn)
    except ZeroDivisionError:
        recall = 0.0
    if (precision + recall) == 0:
        f1_score = 0.0
    else:
        f1_score = 2 * precision * recall / (precision + recall)

    # Round to 2 decimals for reporting
    precision = round(precision, 2)
    recall    = round(recall, 2)
    f1_score  = round(f1_score, 2)

    return (n_gt, n_detected, tp, fp, fn, precision, recall, f1_score)


In [38]:

def run_evaluation_figS1(cutoff=0.8):
    """
    For each subfolder with medoids_<subfolder>.csv,
    read the corresponding ground truth, evaluate using the “Figure S1” method,
    save results in evaluation_figS1_<subfolder>.txt,
    but ONLY for 'Dw' or 'Dkl' subfolders.
    """
    for s_val, dist, noise, run_name, subfolder_path in traverse_folders(
        root_dir=RESULTS_CSV_DIR,
        s_values=S_FOLDERS,
        distances=DISTANCES,
        noise_values=NOISE_LEVELS
    ):
        subfolder_name = os.path.basename(subfolder_path)
        
        # Skip any subfolder that is not Dw or Dkl
        if subfolder_name not in ("Dw", "Dkl"):
            continue
        
        # Path to ground-truth signatures
        gt_csv = os.path.join(RESULTS_CSV_DIR, s_val, f"ground_truth_signatures_{s_val}.csv")
        if not os.path.exists(gt_csv):
            continue
        
        true_signatures = pd.read_csv(gt_csv, index_col=0).to_numpy()
        
        # Path to medoids (extracted signatures)
        medoids_csv = os.path.join(subfolder_path, f"medoids_{subfolder_name}.csv")
        if not os.path.exists(medoids_csv):
            continue
        
        est_signatures = pd.read_csv(medoids_csv, header=None).to_numpy()
        
        # Run “Figure S1” evaluation
        eval_res = evaluation_figS1(
            true_sigs=true_signatures,
            est_sigs=est_signatures,
            cutoff=cutoff,
            dist="cos"  # or "cor" if you want correlation
        )
        
        # Unpack results
        n_gt, n_detected, tp, fp, fn, precision, recall, f1_score = eval_res
        
        # Write results
        eval_txt = os.path.join(subfolder_path, f"evaluation_figS1_{subfolder_name}.txt")
        with open(eval_txt, "w") as f:
            f.write("=== Evaluation Results (Figure S1 Style) ===\n")
            f.write(f"Cutoff: {cutoff}\n")
            f.write(f"s_folder: {s_val}\n")
            f.write(f"distance: {dist}\n")
            f.write(f"noise: {noise}\n")
            f.write(f"run_folder: {run_name}\n")
            f.write(f"subfolder: {subfolder_name}\n")
            f.write(f"Number of Ground Truth: {n_gt}\n")
            f.write(f"Number of Detected: {n_detected}\n")
            f.write(f"True Positives: {tp}\n")
            f.write(f"False Positives: {fp}\n")
            f.write(f"False Negatives: {fn}\n")
            f.write(f"Precision: {precision}\n")
            f.write(f"Recall: {recall}\n")
            f.write(f"F1 Score: {f1_score}\n")

        print(f"Figure S1-style evaluation written to {eval_txt}")


In [56]:
run_evaluation_figS1(cutoff=0.7)

Figure S1-style evaluation written to results_csv_clean/s_8/smooth/n_0.02/Results_20250117_154856__s_8_n_0.02__GRCh37_98_86_39_22a_43_96_37_54/Dw/evaluation_figS1_Dw.txt
Figure S1-style evaluation written to results_csv_clean/s_8/smooth/n_0.02/Results_20250117_154856__s_8_n_0.02__GRCh37_98_86_39_22a_43_96_37_54/Dkl/evaluation_figS1_Dkl.txt
Figure S1-style evaluation written to results_csv_clean/s_8/smooth/n_0.04/Results_20250117_155023__s_8_n_0.04__GRCh37_98_86_39_22a_43_96_37_54/Dw/evaluation_figS1_Dw.txt
Figure S1-style evaluation written to results_csv_clean/s_8/smooth/n_0.04/Results_20250117_155023__s_8_n_0.04__GRCh37_98_86_39_22a_43_96_37_54/Dkl/evaluation_figS1_Dkl.txt
Figure S1-style evaluation written to results_csv_clean/s_8/smooth/n_0.06/Results_20250117_155159__s_8_n_0.06__GRCh37_98_86_39_22a_43_96_37_54/Dw/evaluation_figS1_Dw.txt
Figure S1-style evaluation written to results_csv_clean/s_8/smooth/n_0.06/Results_20250117_155159__s_8_n_0.06__GRCh37_98_86_39_22a_43_96_37_54/Dkl

In [57]:
# 4) Save ground-truth signatures from your known “file_map”
COSMIC_SIGNATURES_PATH = "cosmic_signatures/COSMIC_v3.4_SBS_GRCh37.txt"
save_ground_truth_signatures(COSMIC_SIGNATURES_PATH)

# 5) Evaluate
run_evaluation(cutoff=0.7)

Saved ground truth to results_csv_clean/s_8/ground_truth_signatures_s_8.csv
Saved ground truth to results_csv_clean/s_15/ground_truth_signatures_s_15.csv
Saved ground truth to results_csv_clean/s_25/ground_truth_signatures_s_25.csv
Evaluation written to results_csv_clean/s_8/smooth/n_0.02/Results_20250117_154856__s_8_n_0.02__GRCh37_98_86_39_22a_43_96_37_54/Dw/evaluation_Dw.txt
Evaluation written to results_csv_clean/s_8/smooth/n_0.02/Results_20250117_154856__s_8_n_0.02__GRCh37_98_86_39_22a_43_96_37_54/Dkl/evaluation_Dkl.txt
Evaluation written to results_csv_clean/s_8/smooth/n_0.04/Results_20250117_155023__s_8_n_0.04__GRCh37_98_86_39_22a_43_96_37_54/Dw/evaluation_Dw.txt
Evaluation written to results_csv_clean/s_8/smooth/n_0.04/Results_20250117_155023__s_8_n_0.04__GRCh37_98_86_39_22a_43_96_37_54/Dkl/evaluation_Dkl.txt
Evaluation written to results_csv_clean/s_8/smooth/n_0.06/Results_20250117_155159__s_8_n_0.06__GRCh37_98_86_39_22a_43_96_37_54/Dw/evaluation_Dw.txt
Evaluation written to re

In [ ]:

evaluation_figS1()

In [67]:
est_sigs_path = "results_csv_clean/s_8/hamming/n_0.02/Results_20250117_155001_hamming_s_8_n_0.02__GRCh37_98_86_39_22a_43_96_37_54/Dkl/Dkl_0.csv"
est_sigs2_path ="results_csv_clean/s_8/hamming/n_0.02/Results_20250117_155001_hamming_s_8_n_0.02__GRCh37_98_86_39_22a_43_96_37_54/Dkl/medoids_Dkl.csv"

true_sigs_path = "/Users/lorispodevyn/Documents/COURSES/PROJECT_MASTER_1/Code/Mutational_Signatures/results_csv_clean/s_8/ground_truth_signatures_s_8.csv"

# Load CSV data
est_sigs = pd.read_csv(est_sigs_path, header=None).values
est_sigs2 = pd.read_csv(est_sigs2_path, header=None).values
true_sigs = pd.read_csv(true_sigs_path, header=None).values
true_sigs = true_sigs[1:, 1:]


print("Shape of est_sigs:", est_sigs.shape)
print("Shape of true_sigs:", true_sigs.shape)

# Evaluate
results = evaluation_figS1(true_sigs, est_sigs,cutoff=0.5)
results2 = evaluation_figS1(true_sigs, est_sigs2,cutoff=0.5)
# Print all the results
print("Evaluation Results:")
print(f"Number of ground-truth signatures (n_gt): {results[0]}")
print(f"Number of detected signatures (n_detected): {results[1]}")
print(f"True Positives (TP): {results[2]}")
print(f"False Positives (FP): {results[3]}")
print(f"False Negatives (FN): {results[4]}")
print(f"Precision: {results[5]}")
print(f"Recall: {results[6]}")
print(f"F1 Score: {results[7]}")


print("Evaluation Results medois :")
print(f"Number of ground-truth signatures (n_gt): {results2[0]}")
print(f"Number of detected signatures (n_detected): {results2[1]}")
print(f"True Positives (TP): {results2[2]}")
print(f"False Positives (FP): {results2[3]}")
print(f"False Negatives (FN): {results2[4]}")
print(f"Precision: {results2[5]}")
print(f"Recall: {results2[6]}")
print(f"F1 Score: {results2[7]}")

Shape of est_sigs: (96, 8)
Shape of true_sigs: (96, 8)
Evaluation Results:
Number of ground-truth signatures (n_gt): 8
Number of detected signatures (n_detected): 8
True Positives (TP): 8
False Positives (FP): 0
False Negatives (FN): 5
Precision: 1.0
Recall: 0.62
F1 Score: 0.76
Evaluation Results medois :
Number of ground-truth signatures (n_gt): 8
Number of detected signatures (n_detected): 8
True Positives (TP): 8
False Positives (FP): 0
False Negatives (FN): 5
Precision: 1.0
Recall: 0.62
F1 Score: 0.76
